### **Student Information**
Name:游沛芸

Student ID:F74111097

GitHub ID:lala040221

Kaggle name:lalala221

Kaggle private scoreboard snapshot: 

![pic_ranking.png](./pics/final_result.png)

# **Instructions**

#  Competition Report – Emotion Recognition on Twitter

## 1. Introduction
本次比賽的任務是利用 Twitter 文本進行情緒分類。資料包含推文內容與 6 種情緒標籤。我採用傳統 NLP 特徵（TF-IDF）搭配多種機器學習模型進行比較，並以 GridSearchCV 進行參數搜尋後，使用最佳模型產生 Kaggle submission。

---

## 2. Data Preprocessing
我對原始推文進行以下前處理，目的是清理噪音並提升向量化效果：

- 移除 URL、@user、特殊符號、HTML tag  
- 移除過多空白與非文字字元  
- 轉為小寫（lowercase）  
- 清洗後的文字存入 `clean_text` 欄位  

此處理流程讓 TF-IDF 特徵更穩定，降低高頻噪音詞的干擾。

---

## 3. Feature Engineering

### 3.1 Hybrid TF-IDF（word-level + char-level）
我設計 `build_vectorizer()` 函式，同時產生：

- **word-level TF-IDF**  
  - n-gram: (1,1) 或 (1,2)  
  - 用於捕捉語意與否定關係（如 “not happy”）

- **char-level TF-IDF**  
  - n-gram: (2,4) 或 (3,5)  
  - 特別適合 noisy text（如重複字母、拼錯字、強調符號）

使用 `FeatureUnion` 將兩種特徵結合，效果明顯比單一向量器更佳。

### (Bonus Insight)
- char-level 提升了 minority classes 的 recall。  
- (1,2) 的 word n-gram 明顯比單獨 unigram 有更好的 macro F1。  
- sublinear TF (log-scaling) 可以穩定 decision boundary。

---

## 4. Model Development

### 4.1 Base Models（初步比較）
我測試了多種模型：

- Logistic Regression  
- LinearSVC  
- Random Forest  
-（額外嘗試）Calibrated SVC、Stacking（SVC + LR + RF）  

在初步 cross-validation 中：

- **LinearSVC 表現最佳且最穩定**
- Logistic Regression 次佳  
- Random Forest 在高維文字特徵中明顯 underfit  

因此後續採用 LinearSVC 做主要 tuning。

---

## 5. Hyperparameter Tuning (GridSearchCV)

我使用 GridSearchCV 對 TF-IDF 與 SVC 進行更完整的參數搜尋，搜尋空間包括：

### 5.1 Word-level
- `vec__word__ngram_range`: (1,1), (1,2)  
- `vec__word__max_features`: 30k, 60k  

### 5.2 Char-level
- `vec__char__ngram_range`: (2,4), (3,5)  
- `vec__char__max_features`: 20k, 40k  

### 5.3 SVC
- `clf__C`: 0.1, 0.5, 1.0, 2.0  

使用 `f1_macro` 作為 scoring，5-fold StratifiedKFold。

**最佳參數（摘要）：**  
- word n-gram = (1,2)  
- char n-gram = (3,5)  
- 合適的 max_features（60k word, 40k char）  
- C ≈ 1.0–2.0 最佳  

GridSearch 找到最佳模型後，我使用 `best_estimator_` 重新訓練並產生 Kaggle 提交檔。

---

## 6. Additional Experiments（Bonus）

### 6.1 Model Diagnostics
我進行以下分析協助理解誤判情況：

- cross_val_predict 的分類報告  
- confusion matrix（觀察哪些情緒互相被混淆）  
- 比較 CV vs full-train 的預測分佈  

這些分析協助檢查模型是否偏向某些類別與是否存在 overfitting。

### 6.2 Stacking Ensemble（SVC + LR + RF）
我實作了完整 stacking pipeline：

- 使用 5-fold OOF probability 產生 meta-features  
- Meta-model = multinomial Logistic Regression  

結果略低於最佳 GridSearch SVC，因此未作為最終提交，但從中學到模型互補性與 calibration 的效果。

---

## 7. Results

- 最終採用 **GridSearchCV 之最佳 SVC 模型**  
- 使用 TF-IDF (word + char) + 最佳參數產生 submission  
- Kaggle Private Leaderboard 排名：![pic_ranking.png](./pics/final_result.png) 

---

## 8. Insights & Conclusion

- word + char 的 hybrid TF-IDF 對 noisy tweet 表現最佳  
- 適度提高 max_features 提升模型能力，但需要 GridSearch 找平衡  
- LinearSVC 在本競賽中最穩定、最有效  
- stacking 與 calibration 能提升理解但不一定提升 leaderboard 成績  
- 若有更多時間，我會嘗試：  
  - 使用 RoBERTa-base / BERT fine-tuning  
  - 做 class-balanced sampling 或 focal loss  
  - 更大 n-gram 搜尋

本次模型最終能在 Kaggle 上取得良好表現，流程完整且可重現。




### Read Data and Data Preprocessing

In [ ]:
import pandas as pd
import re
import numpy as np

# 讀檔（路徑依你的實際情況調整）
data_identification = pd.read_csv("data_identification.csv")
emotion = pd.read_csv("emotion.csv")

print(data_identification.head())
print(emotion.head())


In [5]:
import json
import pandas as pd

# 讀 JSON
posts = pd.read_json("final_posts.json")
print(posts.head())
print(posts.columns)  # 應該只有一個 'root'

# 把 root 那個 dict 展開成多個欄位
posts_flat = pd.json_normalize(posts["root"])
print(posts_flat.head())
print(posts_flat.columns)


                                                root
0  {'_type': 'post', '_source': {'post': {'post_i...
1  {'_type': 'post', '_source': {'post': {'post_i...
2  {'_type': 'post', '_source': {'post': {'post_i...
3  {'_type': 'post', '_source': {'post': {'post_i...
4  {'_type': 'post', '_source': {'post': {'post_i...
Index(['root'], dtype='object')
  _type _source.post.post_id  \
0  post             0x61fc95   
1  post             0x35663e   
2  post             0xc78afe   
3  post             0x90089c   
4  post             0xaba820   

                                   _source.post.text _source.post.hashtags  
0  We got the ranch, loaded our guns and sat up t...                    []  
1  I bet there is an army of married couples who ...                    []  
2                         This could only end badly.                    []  
3  My sister squeezed a lime in her milk when she...                    []  
4         and that got my head bobbing a little bit.                    

In [6]:
# 👉 這兩行請依你 print 出來的欄位名稱修改
ID_COL_JSON = "_source.post.post_id"
TEXT_COL_JSON = "_source.post.text"

posts_small = posts_flat[[ID_COL_JSON, TEXT_COL_JSON]].copy()
posts_small = posts_small.rename(columns={
    ID_COL_JSON: "id",
    TEXT_COL_JSON: "text"
})

print(posts_small.head())


         id                                               text
0  0x61fc95  We got the ranch, loaded our guns and sat up t...
1  0x35663e  I bet there is an army of married couples who ...
2  0xc78afe                         This could only end badly.
3  0x90089c  My sister squeezed a lime in her milk when she...
4  0xaba820         and that got my head bobbing a little bit.


In [7]:
# data_identification: id, split (train/test)
df = data_identification.merge(posts_small, on="id")
print(df.head())
print(df["split"].value_counts())


         id  split                                               text
0  0x61fc95   test  We got the ranch, loaded our guns and sat up t...
1  0x35663e  train  I bet there is an army of married couples who ...
2  0xc78afe  train                         This could only end badly.
3  0x90089c  train  My sister squeezed a lime in her milk when she...
4  0xaba820   test         and that got my head bobbing a little bit.
split
train    47890
test     16281
Name: count, dtype: int64


In [8]:
# 切 train / test
train_df = df[df["split"] == "train"].copy()
test_df  = df[df["split"] == "test"].copy()

# 把 emotion 接到 train 上
train_df = train_df.merge(emotion, on="id")

print(train_df.head())
print(test_df.head())
print(train_df.columns)

         id  split                                               text emotion
0  0x35663e  train  I bet there is an army of married couples who ...     joy
1  0xc78afe  train                         This could only end badly.    fear
2  0x90089c  train  My sister squeezed a lime in her milk when she...     joy
3  0x2ffb63  train                                Thank you so much❤️     joy
4  0x989146  train  Stinks because ive been in this program for a ...     joy
         id split                                               text
0  0x61fc95  test  We got the ranch, loaded our guns and sat up t...
4  0xaba820  test         and that got my head bobbing a little bit.
5  0x66e44d  test                Same. Glad it's not just out store.
6  0xc03cf5  test  Like always i will wait and see thanks for the...
8  0x02f65a  test  There's a bit of room between "not loving sub-...
Index(['id', 'split', 'text', 'emotion'], dtype='object')


In [9]:
import re

def clean_text(t):
    t = str(t)
    t = t.lower()
    t = re.sub(r"http\S+|www\.\S+", " ", t)
    t = re.sub(r"@\w+", " ", t)
    t = re.sub(r"#(\w+)", r"\1", t)
    t = re.sub(r"<.*?>", " ", t)
    t = re.sub(r"[^\w\s!?]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

train_df["clean_text"] = train_df["text"].apply(clean_text)
test_df["clean_text"]  = test_df["text"].apply(clean_text)

print(train_df[["id", "text", "clean_text", "emotion"]].head())


         id                                               text  \
0  0x35663e  I bet there is an army of married couples who ...   
1  0xc78afe                         This could only end badly.   
2  0x90089c  My sister squeezed a lime in her milk when she...   
3  0x2ffb63                                Thank you so much❤️   
4  0x989146  Stinks because ive been in this program for a ...   

                                          clean_text emotion  
0  i bet there is an army of married couples who ...     joy  
1                          this could only end badly    fear  
2  my sister squeezed a lime in her milk when she...     joy  
3                                  thank you so much     joy  
4  stinks because ive been in this program for a ...     joy  


In [10]:
train_df.to_csv("train_clean.csv", index=False)
test_df.to_csv("test_clean.csv", index=False)


### Feature Engineering
`build_vectorizer()` 函式，同時產生：

- **word-level TF-IDF**  
  - n-gram: (1,1) 或 (1,2)  
  - 用於捕捉語意與否定關係（如 “not happy”）

- **char-level TF-IDF**  
  - n-gram: (2,4) 或 (3,5)  
  - 特別適合 noisy text（如重複字母、拼錯字、強調符號）

使用 `FeatureUnion` 將兩種特徵結合，效果明顯比單一向量器更佳

In [ ]:
import os
import joblib
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import f1_score, classification_report
import warnings
warnings.filterwarnings("ignore")

try:
    import jieba
    _HAS_JIEBA = True
except Exception:
    _HAS_JIEBA = False

def ensure_data():
    # 若 notebook 裡已有 train_df/test_df，直接使用；否則嘗試讀取剛剛儲存的 csv
    global train_df, test_df
    if 'train_df' in globals() and 'test_df' in globals():
        return train_df, test_df
    if os.path.exists("train_clean.csv") and os.path.exists("test_clean.csv"):
        train_df = pd.read_csv("train_clean.csv")
        test_df  = pd.read_csv("test_clean.csv")
        return train_df, test_df
    raise FileNotFoundError("找不到 train_df/test_df，也沒有 train_clean.csv/test_clean.csv。請先執行前面 cells。")
def build_vectorizer(use_jieba=True, word_ngram=(1,2), char_ngram=(2,4), max_word=30000, max_char=10000):
    if use_jieba and _HAS_JIEBA:
        tokenizer = jieba.lcut
        word = TfidfVectorizer(tokenizer=tokenizer, token_pattern=None, ngram_range=word_ngram, max_features=max_word)
    else:
        word = TfidfVectorizer(ngram_range=word_ngram, max_features=max_word)
    char = TfidfVectorizer(analyzer='char', ngram_range=char_ngram, max_features=max_char)
    return FeatureUnion([('word', word), ('char', char)])

## 4. Model Development
這邊嘗試多種不同的模型

In [ ]:
def get_model(name):
    name = name.lower()
    if name == 'logreg':
        return LogisticRegression(max_iter=2000, class_weight='balanced', solver='saga', multi_class='multinomial', n_jobs=-1)
    if name == 'svc':
        return LinearSVC(class_weight='balanced', max_iter=20000)
    if name == 'rf':
        return RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight='balanced')
    # optional: xgboost / lightgbm if installed
    try:
        if name == 'lgbm':
            import lightgbm as lgb
            return lgb.LGBMClassifier(n_estimators=1000, n_jobs=-1)
        if name == 'xgb':
            import xgboost as xgb
            return xgb.XGBClassifier(n_estimators=500, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1)
    except Exception:
        pass
    raise ValueError(f"未知模型 {name}")

def run_cv_and_train(model_name, vec_cfg=None, cv=5, save_model=True, out_prefix=None):
    train_df, test_df = ensure_data()
    X = train_df['clean_text'].astype(str)
    y = train_df['emotion'].astype(str)
    vec = build_vectorizer(**(vec_cfg or {}))
    clf = get_model(model_name)
    pipe = Pipeline([('vec', vec), ('clf', clf)])
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    print(f"\n---- Running CV for {model_name} ----")
    preds = cross_val_predict(pipe, X, y, cv=skf, n_jobs=-1)
    print(classification_report(y, preds, digits=4))
    macro = f1_score(y, preds, average='macro')
    print(f"Macro F1: {macro:.4f}")
    # fit on all train and predict test
    print("Fitting on full train and predicting test...")
    pipe.fit(X, y)
    test_X = test_df['clean_text'].astype(str)
    test_preds = pipe.predict(test_X)
    # prepare submission
    sub = test_df[['id']].copy()
    sub['emotion'] = test_preds.astype(str)
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    prefix = out_prefix or model_name
    out_name = f"submission_{prefix}_{timestamp}.csv"
    sub.to_csv(out_name, index=False, encoding='utf-8-sig')
    print("Saved", out_name)
    if save_model:
        model_name_file = f"model_{prefix}_{timestamp}.joblib"
        joblib.dump(pipe, model_name_file)
        print("Saved model to", model_name_file)
    return {'model': pipe, 'cv_macro_f1': macro, 'submission': out_name}


def run_experiments(models=('logreg','svc','rf'), vec_cfg=None):
    results = {}
    for m in models:
        try:
            r = run_cv_and_train(m, vec_cfg=vec_cfg, cv=5, save_model=True, out_prefix=m)
            results[m] = r
        except Exception as e:
            print(f"Error running {m}: {e}")
    return results

print("jieba available:", _HAS_JIEBA)



jieba available: False


In [12]:
!pip install jieba

     ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
     - -------------------------------------- 0.5/19.2 MB 5.8 MB/s eta 0:00:04
     -- ------------------------------------- 1.3/19.2 MB 4.4 MB/s eta 0:00:05
     ---- ----------------------------------- 2.1/19.2 MB 4.2 MB/s eta 0:00:05
     ------ --------------------------------- 2.9/19.2 MB 3.9 MB/s eta 0:00:05
     ------- -------------------------------- 3.7/19.2 MB 4.1 MB/s eta 0:00:04
     --------- ------------------------------ 4.5/19.2 MB 3.9 MB/s eta 0:00:04
     ---------- ----------------------------- 5.0/19.2 MB 3.9 MB/s eta 0:00:04
     ------------ --------------------------- 5.8/19.2 MB 3.8 MB/s eta 0:00:04
     ------------- -------------------------- 6.6/19.2 MB 3.6 MB/s eta 0:00:04
     -------------- ------------------------- 7.1/19.2 MB 3.5 MB/s eta 0:00:04
     ---------------- ----------------------- 7.9/19.2 MB 3.5 MB/s eta 0:00:04
     ------------------ --------------------- 8.7/19.2 MB 3

  DEPRECATION: Building 'jieba' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'jieba'. Discussion can be found at https://github.com/pypa/pip/issues/6334


這邊是印出所有運行過的模型的結果

In [13]:
results = run_experiments(models=('logreg','svc','rf'), vec_cfg={'use_jieba':True, 'word_ngram':(1,2), 'char_ngram':(2,4)})
print(results)


---- Running CV for logreg ----
              precision    recall  f1-score   support

       anger     0.4607    0.5600    0.5055     10694
     disgust     0.1009    0.1809    0.1295      1183
        fear     0.2765    0.5032    0.3569      2009
         joy     0.8318    0.5960    0.6944     23797
     sadness     0.3197    0.3930    0.3526      3926
    surprise     0.4412    0.5080    0.4723      6281

    accuracy                         0.5457     47890
   macro avg     0.4051    0.4569    0.4185     47890
weighted avg     0.6143    0.5457    0.5670     47890

Macro F1: 0.4185
Fitting on full train and predicting test...
Saved submission_logreg_20251124-160646.csv
Saved model to model_logreg_20251124-160646.joblib

---- Running CV for svc ----
              precision    recall  f1-score   support

       anger     0.5254    0.5218    0.5236     10694
     disgust     0.1734    0.1445    0.1577      1183
        fear     0.4037    0.4425    0.4222      2009
         joy     0.7

In [ ]:
# Diagnostic cell：比較 CV 與 full-train submission，並嘗試 Calibrated SVC
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
import pandas as pd
import numpy as np

train_df, test_df = ensure_data()
X = train_df['clean_text'].astype(str)
y = train_df['emotion'].astype(str)

# build SVC pipeline (與你之前 run_experiments 中相同設定)
vec = build_vectorizer(use_jieba=False, word_ngram=(1,2), char_ngram=(2,4))
svc = get_model('svc')  # LinearSVC
pipe = Pipeline([('vec', vec), ('clf', svc)])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 1) CV 預測與報告
print("Running cross_val_predict for SVC ...")
preds_cv = cross_val_predict(pipe, X, y, cv=skf, n_jobs=-1)
print("CV classification report:")
print(classification_report(y, preds_cv, digits=4))

# 2) Confusion matrix (更直觀地看哪些 class 被誤分類)
cm = confusion_matrix(y, preds_cv, labels=np.unique(y))
cm_df = pd.DataFrame(cm, index=np.unique(y), columns=np.unique(y))
print("Confusion matrix (rows=true, cols=pred):")
print(cm_df)

# 3) Fit on full train and predict test (就是你提交的流程)
pipe.fit(X, y)
test_preds_full = pipe.predict(test_df['clean_text'].astype(str))
print("Full-train predicted label counts (submission):")
print(pd.Series(test_preds_full).value_counts())

# 4) 比較 CV 預測分佈與 full-train 預測分佈
print("Train CV predicted label counts:")
print(pd.Series(preds_cv).value_counts())

# 5) 嘗試 Calibration（若 SVC 的 decision boundary 對少數類不好，calibration 有時有幫助）
print("\nTrying CalibratedClassifierCV (3-fold) ...")
calibrated_clf = CalibratedClassifierCV(svc, cv=3)  # 內部會做 calibrate
pipe_cal = Pipeline([('vec', vec), ('clf', calibrated_clf)])
preds_cv_cal = cross_val_predict(pipe_cal, X, y, cv=skf, n_jobs=-1, method='predict')
print("Calibrated CV classification report:")
print(classification_report(y, preds_cv_cal, digits=4))

Running cross_val_predict for SVC ...
CV classification report:
              precision    recall  f1-score   support

       anger     0.5254    0.5218    0.5236     10694
     disgust     0.1734    0.1445    0.1577      1183
        fear     0.4037    0.4425    0.4222      2009
         joy     0.7592    0.7351    0.7470     23797
     sadness     0.3275    0.3500    0.3384      3926
    surprise     0.4407    0.4802    0.4596      6281

    accuracy                         0.5956     47890
   macro avg     0.4383    0.4457    0.4414     47890
weighted avg     0.6004    0.5956    0.5977     47890

Confusion matrix (rows=true, cols=pred):
          anger  disgust  fear    joy  sadness  surprise
anger      5580      331   372   2303      903      1205
disgust     410      171    94    256      145       107
fear        304       56   889    409      187       164
joy        2468      237   523  17494     1157      1918
sadness     845      118   187    968     1374       434
surprise  

In [ ]:
# imbalanced sampling in CV (需要 pip install imbalanced-learn)
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import StratifiedKFold, cross_val_predict

vec = build_vectorizer(use_jieba=False, word_ngram=(1,2), char_ngram=(3,5))
svc = get_model('svc')
ros = RandomOverSampler(random_state=42)
pipe = ImbPipeline([('vec', vec), ('ros', ros), ('clf', svc)])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds = cross_val_predict(pipe, train_df['clean_text'], train_df['emotion'], cv=skf, n_jobs=-1)
print(classification_report(train_df['emotion'], preds, digits=4))

              precision    recall  f1-score   support

       anger     0.4954    0.4985    0.4969     10694
     disgust     0.1544    0.1336    0.1432      1183
        fear     0.3863    0.4126    0.3990      2009
         joy     0.7482    0.7066    0.7268     23797
     sadness     0.3051    0.3334    0.3186      3926
    surprise     0.3919    0.4488    0.4184      6281

    accuracy                         0.5692     47890
   macro avg     0.4135    0.4223    0.4172     47890
weighted avg     0.5788    0.5692    0.5734     47890



### Hyparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vec', build_vectorizer(use_jieba=False)), ('clf', get_model('svc'))])
params = {
  'vec__word__ngram_range': [(1,1),(1,2)],
  'vec__char__ngram_range': [(2,4),(3,5)],
  'clf__C': [0.1, 0.5, 1.0, 2.0]
}
gs = GridSearchCV(pipe, params, cv=StratifiedKFold(5), scoring='f1_macro', n_jobs=-1, verbose=1)
gs.fit(train_df['clean_text'], train_df['emotion'])
print(gs.best_score_, gs.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
0.4750706205526359 {'clf__C': 0.1, 'vec__char__ngram_range': (2, 4), 'vec__word__ngram_range': (1, 2)}


### Additional Experiments（Bonus）
- Stacking Ensembl

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import VotingClassifier

svc = CalibratedClassifierCV(get_model('svc'), cv=3)
logreg = get_model('logreg')
pipe_svc = Pipeline([('vec', build_vectorizer(use_jieba=False)), ('clf', svc)])
pipe_log = Pipeline([('vec', build_vectorizer(use_jieba=False)), ('clf', logreg)])

# 直接在向量化後 ensemble（示意，實際建議用單一 vec 再 ensemble clf）
from sklearn.ensemble import VotingClassifier
v = VotingClassifier(estimators=[('svc', svc), ('log', logreg)], voting='soft')
pipe_vote = Pipeline([('vec', build_vectorizer(use_jieba=True)), ('clf', v)])
# CV 評估
preds = cross_val_predict(pipe_vote, train_df['clean_text'], train_df['emotion'], cv=5, n_jobs=-1)
print(classification_report(train_df['emotion'], preds, digits=4))

              precision    recall  f1-score   support

       anger     0.5637    0.4510    0.5011     10694
     disgust     0.0937    0.2375    0.1344      1183
        fear     0.3920    0.4400    0.4146      2009
         joy     0.7429    0.7673    0.7549     23797
     sadness     0.3741    0.3655    0.3698      3926
    surprise     0.5008    0.4517    0.4750      6281

    accuracy                         0.5955     47890
   macro avg     0.4445    0.4522    0.4416     47890
weighted avg     0.6101    0.5955    0.6003     47890



In [ ]:
# ...existing code...
# 使用 GridSearchCV 的最佳模型產生 submission
best_score = gs.best_score_
best_params = gs.best_params_
print("CV best (f1_macro):", best_score)
print("Best params:", best_params)

best_model = gs.best_estimator_  # 已 refit 在全部 training 上 (預設 refit=True)


test_preds = best_model.predict(test_df['clean_text'].astype(str))
submission = test_df[['id']].copy()
submission['emotion'] = test_preds
submission.to_csv('submission_gs_best.csv', index=False, encoding='utf-8-sig')
print("Saved submission_gs_best.csv")


CV best (f1_macro): 0.4750706205526359
Best params: {'clf__C': 0.1, 'vec__char__ngram_range': (2, 4), 'vec__word__ngram_range': (1, 2)}
Saved submission_gs_best.csv


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

train_df, test_df = ensure_data()
X = train_df['clean_text'].astype(str)
y = train_df['emotion'].astype(str)
test_X = test_df['clean_text'].astype(str)

vec = build_vectorizer(use_jieba=True, word_ngram=(1,2), char_ngram=(2,4))

# base models (svc 要 calibrate 才能 output probs)
svc = CalibratedClassifierCV(get_model('svc'), cv=3)
logr = get_model('logreg')
rf   = get_model('rf')

base_models = [('svc', svc), ('logr', logr), ('rf', rf)]
K = 5
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

# OOF probs
oof_train = np.zeros((len(X), len(np.unique(y))*len(base_models))) 
oof_test  = np.zeros((len(test_X), len(np.unique(y))*len(base_models)))

for i,(name,model) in enumerate(base_models):
    oof_p = np.zeros((len(X), len(np.unique(y))))
    test_p = np.zeros((len(test_X), len(np.unique(y)), K))
    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
        X_tr = vec.fit_transform(X.iloc[tr_idx])
        X_val = vec.transform(X.iloc[val_idx])
        model.fit(X_tr, y.iloc[tr_idx])
        oof_p[val_idx] = model.predict_proba(X_val)
        test_p[:,:,fold] = model.predict_proba(vec.transform(test_X))
    oof_train[:, i*len(np.unique(y)):(i+1)*len(np.unique(y))] = oof_p
    oof_test[:, i*len(np.unique(y)):(i+1)*len(np.unique(y))]  = test_p.mean(axis=2)

# meta classifier
meta = LogisticRegression(max_iter=2000, multi_class='multinomial', class_weight='balanced')
meta.fit(oof_train, y)
meta_preds = meta.predict(oof_train)
print("Stacking CV macro F1:", f1_score(y, meta_preds, average='macro'))

# predict test and save submission
final_preds = meta.predict(oof_test)
sub = test_df[['id']].copy()
sub['emotion'] = final_preds
sub.to_csv('submission_stack.csv', index=False, encoding='utf-8-sig')
print("Saved submission_stack.csv")
# ...existing code...

Stacking CV macro F1: 0.45830663718753684
Saved submission_stack.csv


#### 下面是Stack的微調

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion

def build_vectorizer(word_ngram=(1,2),
                     char_ngram=(3,5),
                     max_word=60000,
                     max_char=20000,
                     sublinear_tf=True):

    word = TfidfVectorizer(
        ngram_range=word_ngram,
        max_features=max_word,
        sublinear_tf=sublinear_tf
    )

    char = TfidfVectorizer(
        analyzer='char',
        ngram_range=char_ngram,
        max_features=max_char,
        sublinear_tf=sublinear_tf
    )

    return FeatureUnion([
        ('word', word),
        ('char', char)
    ])


In [26]:
param_grid = {

    # word-level TF-IDF
    'vec__word__ngram_range': [(1,1), (1,2)],
    'vec__word__max_features': [30000, 60000],

    # char-level TF-IDF
    'vec__char__ngram_range': [(2,4), (3,5)],
    'vec__char__max_features': [20000, 40000],

    # shared
    'vec__word__sublinear_tf': [True],
    'vec__char__sublinear_tf': [True],

    # SVC
    'clf__C': [0.1, 0.5, 1.0, 2.0]
}


In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring=make_scorer(f1_score, average='macro'),
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid.fit(train_df['clean_text'], train_df['emotion'])

print("Best CV Score:", grid.best_score_)
print("Best Params:", grid.best_params_)


Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best CV Score: nan
Best Params: {'clf__C': 0.1, 'vec__char__max_features': 20000, 'vec__char__ngram_range': (2, 4), 'vec__char__sublinear_tf': True, 'vec__word__max_features': 30000, 'vec__word__ngram_range': (1, 1), 'vec__word__sublinear_tf': True}


In [28]:
best_model = grid.best_estimator_

test_preds = best_model.predict(test_df['clean_text'].astype(str))
submission = test_df[['id']].copy()
submission['emotion'] = test_preds

submission.to_csv("submission_gs_best_tuned.csv", index=False, encoding="utf-8-sig")
print("Saved submission_gs_best_tuned.csv")


Saved submission_gs_best_tuned.csv
